In [ ]:
## other functions ##
## not used anymore, but logic might be useful

## creating the final df, automatically creating group names & global experiment structure, not needed anymore due to creation in create_df()


# xs code maybe to be used later for network analysis for suite2p dataframe / csv creation
#     ImgShape = getImg(suite2p_dict["ops"]).shape
#     ImgShape = f"[{ImgShape[0]},{ImgShape[1]}]"
#     ImgShape = [ImgShape] * len(suite2p_dict["IsUsed"])
#     df = pd.DataFrame({"experiment_date": suite2p_dict["experiment_date"],
#                        "IsUsed": suite2p_dict["IsUsed"],
#                        "Skew": suite2p_dict["stat"]["skew"],
#                        "ImgShape": ImgShape,

#     # suite2p_dict["experiment_date"] = int(os.path.split(data_folder)[1][8:14]) #TODO change if needed in future
#     # suite2p_dict["file_name"] = os.path.split(data_folder)[1]

###DUPLICATE

def get_all_pkl_outputs_in_path(path):
    processed_files = []
    file_names = []
    for current_path, directories, files in os.walk(path):
        for file_name in files:
            full_path = os.path.join(current_path, file_name)
            processed_files.append(full_path)
        file_names.append(files)
    return processed_files, file_names[0]

def find_pkl_indices(main_folder, group):
    predictions_deltaF_files = get_file_name_list(main_folder, "predictions_deltaF.npy",supress_printing = True)
    pkl_files = get_pkl_file_name_list(main_folder)
    group_pkl_list = []
    for file in predictions_deltaF_files:
        if str(group) in file:
            for i in range(len(pkl_files)):
                if str(file[len(group)+1:-38]) in pkl_files[i]:
                    group_pkl_list.append(i)
    return group_pkl_list

def pkl_processing(main_folder, groups): #creates a list of indices that belong to given group in list "pkl_files"
    pkl_files = get_pkl_file_name_list(main_folder)
    predictions_deltaF_files = get_file_name_list(main_folder, "predictions_deltaF.npy", supress_printing = True)
    if len(predictions_deltaF_files) == len(pkl_files):
        print(f"{len(pkl_files)} .pkl files found:")
        print(pkl_files)
        pkl_is_group = {}
        for i, group in enumerate(groups):
            pkl_is_group[f"pkl_is_g{i+1}"] = find_pkl_indices(main_folder, group)
    else:
        print("the amount of .pkl files doen't match the amopunt of cascade predicition files, please double check")
    return pkl_is_group

def create_global_exp_structure(main_folder, groups):
    pkl_files = get_pkl_file_name_list(main_folder)
    pkl_is_group = pkl_processing(main_folder, groups)
    global_exp_structure = {}
    for i, group in enumerate(groups):
        group_name = str(group[len(main_folder)+1:])
        global_exp_structure[group_name] = { ## add more sub-samples (like "smaple1") dynamically, maybe by adding sub sample size to configurations file, adding another for loop? Necessarry? what does data strcture usually look like?##
            'sample1':[pkl_files[i2] for i2 in pkl_is_group[f"pkl_is_g{i+1}"]],}
    return global_exp_structure

def build_experiment_dfs(input_path, experiment_structure):

    """Builds out the dataframes from pkl files as described in suite2p_utility (creates csv) and analysis_utility (to create pkl)"""
    experiment_cell_stats, experiment_binned_stats = pd.DataFrame(), pd.DataFrame()

    for group in experiment_structure.keys():
        for dataset in experiment_structure[group].keys():
            for file_name in experiment_structure[group][dataset]:
                file_path = os.path.join(input_path, file_name)
                if os.path.isfile(file_path):
                    try:
                        file_dict = pd.read_pickle(file_path)
                        cell_stats, binned_stats = file_dict["cell_stats"], file_dict["binned_stats"]
                
                        for stats, experiment_stats in zip((cell_stats, binned_stats), 
                                                   (experiment_cell_stats, experiment_binned_stats)):
                            stats[["group", "dataset", "file_name"]] = group, dataset, file_name
                            experiment_cell_stats = pd.concat((experiment_cell_stats, cell_stats))
                            experiment_binned_stats = pd.concat((experiment_binned_stats, binned_stats))
                    except Exception as e:
                        print(f"Error processing file {file_name}: {e}")
                else:
                    print(f"File not found: {file_name}")

    return experiment_cell_stats, experiment_binned_stats

## translate files to .pkl ###

# def list_all_files_of_type(input_path, filetype):
#     return [file for file in os.listdir(input_path) if file.endswith(filetype)]

def spike_df_iterator(input_path, return_name=True):
    """Finds all csv files in a directory and begins creating a pd.DataFrame for a pickle file"""
    for csv_file in list_all_files_of_type(input_path, "csv"):
        csv_path = os.path.join(input_path, csv_file)
        csv_df = pd.read_csv(csv_path)
        yield csv_df, csv_file if return_name else csv_df

def spike_list_translator(input_string):
    """This funciton is nested in the next. It is designed to convert the time stamp of each event into a time
        during the experiment (e.g. frame 2 = 1.1 seconds into the recording)"""
    string_list = string_to_list_translator(input_string)
    return np.array(string_list).astype(int) * FRAME_INTERVAL

# def amplitude_list_translator(input_string): ## removed for now
#     """Translates csv Amplitude values into pd.DataFrame friendly format"""
#     amp_string_list = string_to_list_translator(input_string)
#     amp_string_list = np.array(amp_string_list).astype(float)
#     return np.around(amp_string_list)

def string_to_list_translator(input_string, strip_before_split="[ ]", split_on=" "):
    split_string = input_string.strip(strip_before_split).split(split_on)
    return list(filter(None, split_string))

def calculate_cell_stats(input_df, calculate_freq=True):
    """Puts all of the above calculations into a output dataframe"""
    output_df = input_df.copy()
    if calculate_freq:
        output_df = calculate_cell_freq(output_df)
    
    return output_df

def calculate_cell_freq(input_df):
    """Calculate individual cell frequencies"""
    output_df = input_df.copy()
    output_df["SpikesFreq"] = output_df["EstimatedSpikes"] / ((input_df["Total Frames"]-64) / frame_rate) ## -64 because first and last entries in cascade are NaN, thus not considered in estimated spikes
    return output_df

def calculate_binned_stats(input_df):
    """Basic method of measuring population spikes and events"""
    local_df = input_df.copy()

    bins = np.arange(0, EXPERIMENT_DURATION + BIN_WIDTH, BIN_WIDTH)
    # population_spikes, _ = np.histogram(np.hstack(local_df["PeakTimes"].values), bins=bins) ### peak times removed
    # population_frequency = population_spikes / BIN_WIDTH

    bin_stats = pd.DataFrame.from_dict({
        "Bin_Limits": [(bins[bin_index], bins[bin_index + 1]) for bin_index in range(len(bins) - 1)]})
        #, "Spikes": population_spikes,
        # "Frequency": population_frequency})
        
    return bin_stats

def process_spike_csvs_to_pkl(input_path, output_path, overwrite=False):  ### alternatively if we cant find a ways to fix the issue with the .pkl having too many rows we could just add everything in the csv already and then do pd.to_pickle()
    """This will convert .csv files into pickle files which behave like dataframes; but are faster and preserve CPU RAM"""
    for spike_df, file_name in spike_df_iterator(input_path):
        processed_path = os.path.join(output_path, 
                                      f"{os.path.splitext(file_name)[0]}"
                                      f"Dur{int(EXPERIMENT_DURATION)}s"
                                      f"Int{int(FRAME_INTERVAL*1000)}ms"
                                      f"Bin{int(BIN_WIDTH*1000)}ms"
                                        + ("_filtered" if FILTER_NEURONS else "") +
                                      ".pkl")

        if os.path.exists(processed_path) and not overwrite:
            print(f"Processed file {processed_path} already exists!")
            continue
            
        if FILTER_NEURONS:
            spike_df = spike_df[spike_df["IsUsed"]]
        try:
            processed_dict = {
                "cell_stats": calculate_cell_stats(spike_df),
                "binned_stats": calculate_binned_stats(spike_df)}
        

            pd.to_pickle(processed_dict, processed_path)
        except ValueError as e:
            if "need at least one array to concatenate" in str(e):
                print(f"Error processing {file_name}: {e}")
            else:
                raise

    print(f".pkl files saved under {main_folder+r'/pkl_files'}")

##

def get_active_proportion_list(prediction_file, relative_values = True):
    prediction_array = np.load(rf"{prediction_file}", allow_pickle=True)
    proportion_list = []
    for time in range(prediction_array.shape[1]):
        active_proportion = 0
        for entry in prediction_array[:,time]:
            if entry !=0 and not np.isnan(entry):
                active_proportion = active_proportion+1
        if relative_values:
            proportion_list.append(active_proportion/len(prediction_array))
        else:
            proportion_list.append(active_proportion)
    return proportion_list

